In [1]:
import os
import traceback
import numpy as np
import numpy.random as npr
import tensorflow as tf
import keras.backend as K
from keras.models import load_model
from keras.layers import Conv2D, TimeDistributed, Flatten, Dense, BatchNormalization
from keras.models import Input, Model
from keras.layers import Layer
from keras.applications.resnet50 import ResNet50
import cv2
import pandas as pd

Using TensorFlow backend.


In [2]:
from utils import bbox_overlaps, bbox_transform,loss_cls, smoothL1, unmap, filter_boxes, clip_boxes, py_cpu_nms

In [3]:
def read_file(filename):  
    col = ['bottomLeftX','bottomLeftY','bottomRightX','bottomRightY','topRightX','topRightY','topLeftX','topLeftY','category','difficult']
    dfr = pd.read_csv(filename,sep=" ",names = col,index_col=None, header=None)
    #print(dfr)
    return dfr

In [4]:
def draw_anchors(img_path, anchors, pad_size=50):
    im = Image.open(img_path)
    w,h=im.size
    a4im = Image.new('RGB',
                    (w+2*pad_size, h+2*pad_size),   # A4 at 72dpi
                    (255, 255, 255))  # White
    a4im.paste(im, (pad_size,pad_size))  # Not centered, top-left corner
    for a in anchors:
        a=(a+pad_size).astype(int).tolist()
        draw = ImageDraw.Draw(a4im)
        draw.rectangle(a,outline=(255,0,0), fill=None)
    return a4im


In [5]:
##### define the ROI Pooling layer
class RoIPooling(Layer):
    def __init__(self, size=(7, 7)):
        self.size = size
        super(RoIPooling, self).__init__()

    def build(self, input_shape):
        self.shape = input_shape
        super(RoIPooling, self).build(input_shape)

    def call(self, inputs, **kwargs):
        ind=K.reshape(inputs[2],(-1,))
        x = K.tf.image.crop_and_resize(inputs[0], inputs[1], ind, self.size)
        return x

    def compute_output_shape(self, input_shape):
        a=input_shape[1][0]
        b=self.size[0]
        c=self.size[1]
        d=input_shape[0][3]
        return (a,b,c,d)

In [6]:
######## the RCNN model - change output shape based on number of dota classes
def RCNN():
    feature_map = Input(batch_shape=(None,None,None,2048)) # can use multiple feature maps (a map for every roi/or batch) 
    rois = Input(batch_shape=(None, 4))
    ind = Input(batch_shape=(None, 1),dtype='int32')
    p1 = RoIPooling()([feature_map, rois, ind])
    flat1 = Flatten()(p1)
    fc1 = Dense(units=1024,activation="relu",name="fc2")(flat1)

    output_deltas = Dense(units = 4*15,activation="linear",kernel_initializer="uniform",name="deltas2")(fc1)

    output_scores = Dense(units = 1*15,activation="softmax",kernel_initializer="uniform",name="scores2")(fc1)

    model = Model(inputs=[feature_map, rois, ind],outputs=[output_scores,output_deltas])
    
    return model

In [7]:
BATCH = 512

In [8]:
model_rcnn = RCNN()
model_rcnn.compile(optimizer='rmsprop', loss={'deltas2':smoothL1, 'scores2':'categorical_crossentropy'})

In [20]:
############### Prepare MINI Batch

In [9]:
anchor_ratios = [1, 1/2, 2, 1/3, 3, 1/4, 4, 1/5, 5]

In [10]:
FG_FRAC=.25
FG_THRESH=.5
BG_THRESH_HI=.5
BG_THRESH_LO=.1
model_resnet = ResNet50(include_top=False, input_shape = (800,800, 3)) 
rpn_model = load_model('rpnmodel0412.h5', custom_objects={'loss_cls': loss_cls,'smoothL1':smoothL1})

C:\Users\user\Anaconda3\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [11]:
def generate_anchors(ratios, base_width, base_height,scales=np.asarray([3,6,12])):
    """
    Generate anchor (reference) windows by enumerating aspect ratios X
    scales wrt a reference (0, 0, w_stride-1, h_stride-1) window.
    """
    base_anchor = np.array([0, 0, base_width-1, base_height-1])
    ratio_anchors = _ratio_enum(base_anchor, ratios)
    anchors = np.vstack([_scale_enum(ratio_anchors[i, :], scales) for i in range(ratio_anchors.shape[0])]) #shape[0] gives ratio.size number of anchors
    return anchors

def _whctrs(anchor):
    """
    Return width, height, x center, and y center for an anchor (window).
    """
    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    return w, h, x_ctr, y_ctr

def _mkanchors(ws, hs, x_ctr, y_ctr):
    """
    Given a vector of widths (ws) and heights (hs) around a center
    (x_ctr, y_ctr), output a set of anchors (windows).
    """

    ws = ws[:, np.newaxis]
    hs = hs[:, np.newaxis]
    anchors = np.hstack((x_ctr - 0.5 * (ws - 1),
                         y_ctr - 0.5 * (hs - 1),
                         x_ctr + 0.5 * (ws - 1),
                         y_ctr + 0.5 * (hs - 1)))
    return anchors

def _ratio_enum(anchor, ratios):
    """
    Enumerate a set of anchors for each aspect ratio wrt an anchor.
    """

    w, h, x_ctr, y_ctr = _whctrs(anchor)
    size = w * h
    size_ratios = size / ratios
    ws = np.round(np.sqrt(size_ratios))
    hs = np.round(ws * ratios)
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr) # center reamins the same
    return anchors

def _scale_enum(anchor, scales):
    """
    Enumerate a set of anchors for each scale wrt an anchor.
    """

    w, h, x_ctr, y_ctr = _whctrs(anchor)
    ws = w * scales
    hs = h * scales
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    return anchors


In [12]:
def bbox_transform_inv(boxes, deltas):
    if boxes.shape[0] == 0:
        return np.zeros((0, deltas.shape[1]), dtype=deltas.dtype)

    boxes = boxes.astype(deltas.dtype, copy=False)
    #print('boxes.shape-', boxes.shape)
    #print('deltas.shape - ', deltas.shape)
    widths = abs(boxes[:, 2] - boxes[:, 0]) + 1.0
    heights = abs(boxes[:, 3] - boxes[:, 1]) + 1.0
    ctr_x = boxes[:, 0] + 0.5 * widths
    ctr_y = boxes[:, 1] + 0.5 * heights
    dx = deltas[:, 0::4]
    dy = deltas[:, 1::4]
    dw = deltas[:, 2::4]
    dh = deltas[:, 3::4]
    
    inds = min(len(widths),len(dx))
    
    pred_ctr_x = dx[:inds]*widths[:inds,np.newaxis] + ctr_x[:inds, np.newaxis]
    pred_ctr_y =  dy[:inds]*heights[:inds, np.newaxis] + ctr_y[:inds, np.newaxis]
    pred_w = np.exp(dw[:inds])*widths[:inds, np.newaxis]
    pred_h = np.exp(dh[:inds])*heights[:inds, np.newaxis]
    
    #print('pauseeeeeeee')
    
    pred_boxes = np.zeros((inds,4), dtype=deltas.dtype)
    # x1
    pred_boxes[:, 0::4] = pred_ctr_x - 0.5 * pred_w
    # y1
    pred_boxes[:, 1::4] = pred_ctr_y - 0.5 * pred_h
    # x2
    pred_boxes[:, 2::4] = pred_ctr_x + 0.5 * pred_w
    # y2
    pred_boxes[:, 3::4] = pred_ctr_y + 0.5 * pred_h

    return pred_boxes

In [13]:
def clip_boxes(boxes, im_width, im_height):
    """
    Clip boxes to image boundaries.
    """

    # x1 >= 0
    boxes[:, 0::4] = np.maximum(np.minimum(boxes[:, 0::4], im_height - 1), 0)
    # y1 >= 0
    boxes[:, 1::4] = np.maximum(np.minimum(boxes[:, 1::4], im_width - 1), 0)
    # x2 < im_shape[1]
    boxes[:, 2::4] = np.minimum(np.maximum(boxes[:, 2::4], im_height - 1), 0)
    # y2 < im_shape[0]
    boxes[:, 3::4] = np.minimum(np.maximum(boxes[:, 3::4], im_width - 1), 0)
    return boxes

In [29]:
def filter_boxes(boxes, min_size):
    """Remove all boxes with any side smaller than min_size."""
    ws = (boxes[:, 2] - boxes[:, 0] + 1)*10
    hs = (boxes[:, 3] - boxes[:, 1] + 1)*10
    keep = np.where((ws >= min_size) & (hs >= min_size))[0]
    return keep

In [229]:
def parse_(filename,w_scale, h_scale):
    df = read_file(filename)
    width=0
    height=0
    list_of_widths = abs(df['topLeftX']-df['topRightX'])
    list_of_hts = abs(df['topLeftY']-df['bottomLeftY'])
    
    category=[]
    xmin=[]
    ymin=[]
    xmax=[]
    ymax=[]
    for i in df['category']:
        category.append(i)
    
    for j in df['bottomLeftX']:
        xmin.append(int(j)*(w_scale))
    
    for k in df['bottomLeftY']:
        ymin.append(int(k)*(h_scale))
    
    for l in df['topRightX']:
        xmax.append(int(l)*(w_scale))
    
    for m in df['topRightY']:
        ymax.append(int(m)*(h_scale))
        
    gt_boxes=[list(box) for box in zip(xmin,ymin,xmax,ymax)]
    
    return category, np.asarray(gt_boxes, np.float)

In [231]:
def produce_batch(filepath, gt_boxes, category, img):
    
    #img = cv2.imread(filepath)
    img_width = np.shape(img)[1]#*scale[1]
    img_height = np.shape(img)[0]#*scale[0]
    img = np.expand_dims(img, axis=0)
    print('img shape inside produce batch - ', img.shape)
    feature_map = model_resnet.predict(img)
    #print("feature map- ", feature_map.shape)
    height = np.shape(feature_map)[1]
    width = np.shape(feature_map)[2]
    num_feature_map = width*height
    w_stride = img_width/width
    h_stride = img_height/height
    #generate base anchors according output stride.
    #base anchors are 9 anchors wrt a tile (0,0,w_stride-1,h_stride-1)
    base_anchors = generate_anchors(anchor_ratios,w_stride,h_stride)
    #slice tiles according to image size and stride.
    #each 1x1x1532 feature map is mapping to a tile.
    shift_x = np.arange(0, width) * w_stride
    shift_y = np.arange(0, height) * h_stride
    shift_x, shift_y = np.meshgrid(shift_x, shift_y)
    shifts = np.vstack((shift_x.ravel(), shift_y.ravel(), shift_x.ravel(),
                            shift_y.ravel())).transpose()
    #apply base anchors to all tiles, to have a num_feature_map*9 anchors.
    all_anchors = (base_anchors.reshape((1, 27, 4)) +
                    shifts.reshape((1, num_feature_map, 4)).transpose((1, 0, 2)))
    #print(all_anchors.shape)
    total_anchors = num_feature_map*27
    all_anchors = all_anchors.reshape((total_anchors, 4))
    
    # feed feature map to pretrained RPN model, get proposal labels and bboxes.
    
    res = rpn_model.predict(feature_map)
    
    scores = np.array(res[1])
    deltas = np.array(res[0])
    
    scores = scores.reshape(-1,1)
    
    deltas = np.reshape(deltas,(-1,4))
    
    # proposals transform to bbox values (x1, y1, x2, y2)
    proposals = bbox_transform_inv(all_anchors, deltas)
    # (x1,y1,x2,y2)
    proposals = clip_boxes(proposals, img_width,img_height)
    # remove small boxes, here threshold is 40 pixel
    keep = filter_boxes(proposals, 40)#remove boxes with size greater than this returns the indices
    proposals = proposals[keep-1, :]
    scores = scores[np.array(keep-1)] ############################
    #scores = scores[keep]

    # sort socres and only keep top 6000.
    pre_nms_topN=6000
    order = scores.ravel().argsort()[::-1]
    if pre_nms_topN > 0:
        order = order[:pre_nms_topN]
    proposals = proposals[order, :]
    scores = scores[order]
    # apply NMS to to 6000, and then keep top 300
    post_nms_topN=300
    keep = py_cpu_nms(np.hstack((proposals, scores)), 0.7)
    
    if post_nms_topN > 0:
        keep = keep[:post_nms_topN]
    
    # add gt_boxes to proposals.
    proposals=np.vstack( (proposals, gt_boxes) )
    # calculate overlaps of proposal and gt_boxes
    overlaps = bbox_overlaps(proposals, gt_boxes)
    gt_assignment = overlaps.argmax(axis=1)
    max_overlaps = overlaps.max(axis=1)

    # sub sample
    fg_inds = np.where(max_overlaps >= FG_THRESH)[0]
    fg_rois_per_this_image = min(int(BATCH*FG_FRAC), fg_inds.size) #2
    # Sample foreground regions without replacement
    if fg_inds.size > 0:
        fg_inds = npr.choice(fg_inds, size=fg_rois_per_this_image, replace=False)
    bg_inds = np.where((max_overlaps/10 < BG_THRESH_HI) &
                       (max_overlaps/10 >= BG_THRESH_LO))[0]
    bg_rois_per_this_image = BATCH - fg_rois_per_this_image
    bg_rois_per_this_image = min(bg_rois_per_this_image, bg_inds.size)
    # Sample background regions without replacement
    if bg_inds.size > 0:
        bg_inds = npr.choice(bg_inds, size=bg_rois_per_this_image, replace=False)
    # The indices that we're selecting (both fg and bg)
    keep_inds = np.append(fg_inds, bg_inds)
    # Select sampled values from various arrays:
    rois = proposals[keep_inds]
    gt_rois = gt_boxes[gt_assignment[keep_inds]]
    targets = bbox_transform(rois-0.5, gt_rois)#input rois rois-1 to introduce randomness
    print(targets.any())
    rois_num = targets.shape[0]
    batch_box = np.zeros((rois_num, 15, 4))
    for i in range(rois_num):
        batch_box[i, category] = targets[i]
    batch_box = np.reshape(batch_box, (rois_num, -1))
    # get gt category
    batch_categories = np.zeros((rois_num, 15, 1))
    for i in range(rois_num):
        batch_categories[i, category] = 1
    batch_categories = np.reshape(batch_categories, (rois_num, -1))
    
    return rois, batch_box, batch_categories, feature_map

In [235]:
#### prepare data for training
img_path = r'F:\DOTA (Dataset)\Training Set\part 1\test images'
anno_path = r'F:\DOTA (Dataset)\Training Set\part 1\test annos'

import glob
#from multiprocessing import Process, Queue

In [227]:
def preprocess_(file):
    img = cv2.imread(file)
    dim = (800,800)
    img_ = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    scale_w = img_.shape[0]/img.shape[0]
    scale_h = img_.shape[1]/img.shape[1]
    return img_,scale_w, scale_h

In [22]:
def train_generator():
    
    batch_rois=[]
    batch_featuremap_inds=[]
    batch_categories=[]
    batch_bboxes=[]
    fc_index=0 
    
    while 1:
        for filename in os.listdir(anno_path):
            if filename.endswith(".txt"):
                try:
                    file = anno_path + '\\'
                    filepath = img_path + '\\' + filename[:-4]+ '.png'
                    img, scw, sch = preprocess_(filepath)
                    print('image shape is: ', img.shape)
                    category, gt_boxes = parse_(file+filename,scw,sch)
                    if len(gt_boxes)==0:
                        continue
                    rois, bboxes, categories, feature_map = produce_batch(filepath, gt_boxes, category, img)
                except Exception:
                    print('parse label or produce batch failed')
                    traceback.print_exc()
                    continue
                if len(rois)<=0:
                    continue
                for i in range(len(rois)):
                    batch_rois.append(rois[i])
                    batch_featuremap_inds.append(i)
                    batch_categories.append(categories[i])
                    batch_bboxes.append(bboxes[i])
                
                a=feature_map
                b=np.asarray(batch_rois)
                c=np.asarray(batch_featuremap_inds)
                d=np.asarray(batch_categories)
                e=np.asarray(batch_bboxes)
                f=np.zeros((len(rois),a.shape[1],a.shape[2],a.shape[3]))
                f[0]=feature_map[0]
                yield [f,b,c], [d,e]
                batch_rois=[]
                batch_featuremap_inds=[]
                batch_categories=[]
                batch_bboxes=[]
                #fc_index=0

In [243]:
batch_rois=[]
batch_featuremap_inds=[]
batch_categories=[]
batch_bboxes=[]
featuremap = []
fc_index=0
category = 0
for filename in os.listdir(anno_path):
    file = anno_path + '\\'
    filepath = img_path + '\\' + filename[:-4]+ '.png'
    img, scw, sch = preprocess_(filepath)
    
    _, gt_boxes = parse_(file+filename,scw,sch)
    
    
    if len(gt_boxes)==0:
        continue
    
    rois, bboxes, categories, feature_map = produce_batch(filepath, gt_boxes, category, img)
    category = category+1
    #tiles, labels, bboxes = minibatch(filepath, gt_boxes, scale)
    for i in range(len(rois)):
        featuremap.append(feature_map)
        batch_rois.append(rois[i])
        batch_featuremap_inds.append(i)
        batch_categories.append(categories[i])
        batch_bboxes.append(bboxes[i])

a = np.asarray(featuremap)
b = np.asarray(batch_rois)
c = np.asarray(batch_featuremap_inds)
d = np.asarray(batch_categories)
e = np.asarray(batch_bboxes)
f = a#np.zeros((len(rois),a.shape[1],a.shape[2],a.shape[3]))
#f[0] = featuremap[0]

if not a.any() or not b.any() or not c.any() or not d.any() or not e.any() or not f.any():
        print("empty array found.")
print('a.any() - ', a.any(), 'b.any() - ', b.any(), 'c.any() - ', c.any(), 'd.any() - ', d.any(), 'e.any() - ', e.any(), 
     'f.any() - ', f.any())


In [239]:
f.shape, b.shape, c.shape, d.shape, e.shape

((128, 25, 25, 2048), (487, 4), (487, 1), (487, 15), (487, 60))

In [44]:
# part of a previous test
f.shape, b.shape, c.shape, d.shape, e.shape

((3, 25, 25, 2048), (3, 4), (3,), (3, 15), (3, 60))

In [45]:
model_rcnn.fit([f,b,c], [d,e], epochs=10)

Epoch 1/10
3/3 [==============================] - 5s 2s/step - loss: 2.7089 - scores2_loss: 2.7081 - deltas2_loss: 8.0270e-04
Epoch 2/10
3/3 [==============================] - 1s 462ms/step - loss: 2.7028 - scores2_loss: 2.7021 - deltas2_loss: 6.9402e-04
Epoch 3/10
3/3 [==============================] - 1s 467ms/step - loss: 2.6985 - scores2_loss: 2.6979 - deltas2_loss: 6.2454e-04
Epoch 4/10
3/3 [==============================] - 1s 472ms/step - loss: 2.6949 - scores2_loss: 2.6943 - deltas2_loss: 5.7156e-04
Epoch 5/10
3/3 [==============================] - 1s 467ms/step - loss: 2.6916 - scores2_loss: 2.6911 - deltas2_loss: 5.2818e-04
Epoch 6/10
3/3 [==============================] - 1s 466ms/step - loss: 2.6887 - scores2_loss: 2.6882 - deltas2_loss: 4.9124e-04
Epoch 7/10
3/3 [==============================] - 1s 494ms/step - loss: 2.6859 - scores2_loss: 2.6855 - deltas2_loss: 4.5903e-04
Epoch 8/10
3/3 [==============================] - 2s 505ms/step - loss: 2.6833 - scores2_loss: 2.682

In [238]:
c = c.reshape(-1,1)

In [183]:
######## Using input generator
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='./rcnn_weights_2.hdf5', monitor='loss', verbose=1, save_best_only=True)
model_rcnn.fit_generator(train_generator(), steps_per_epoch=30, epochs=10, callbacks=[checkpointer])

In [19]:
#pred_img_path = r'C:\Users\user\Desktop\Abhilash\4th year\DOP\P1053.png'
pred_anno_path = r'F:\DOTA (Dataset)\Training Set\part 1\test annos'
pred_img_path = r'F:\DOTA (Dataset)\Training Set\part 1\test images'

In [26]:
def selectMax(categories):
    classes = ['plane','ship','storage tank','baseball diamond','tennis court','basketball court',
               'ground track field','harbor','bridge','large vehicle','small vehicle','helicopter',
               'roundabout','soccer ball field','swimming pool']
    category = []
    for i in len(categories):
        ind = np.argmax(categories[i])
        category.append(classes[ind[0]])
    return np.asarray(category)

In [25]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.text as text

def predictImages(imgfile, model_rcnn,rois):
    #testimg = cv2.imread(filename)
    testimg = np.expand_dims(imgfile, axis=0)
    pred_resnet = model_resnet.predict(testimg)
    pred_rpn = rpn_model.predict(pred_resnet)
    
    pred_rpn_scores = np.asarray(pred_rpn[1])
    pred_rpn_deltas = np.asarray(pred_rpn[0])
    
    pred_rpn_deltas = pred_rpn_deltas.reshape(-1,4)
    pred_rpn_scores = pred_rpn_scores.reshape(-1,1)
    
    inds = [i for i in range(0,len(rois))]
    pred_rcnn = model_rcnn.predict([pred_resnet, rois, np.asarray(inds)])
    
    pred_deltas = np.asarray(pred_rcnn[1]).reshape(-1,4)
    pred_categories = np.asarray(pred_rcnn[0]).reshape(-1,1)
    
    boxes = bbox_transform_inv(rois, pred_deltas)
    pred_categories = selectMax(pred_categories)
    return pred_categories, boxes

In [222]:
def plotPredicted(im, deltas, categories):
    #im = np.array(Image.open(img), dtype=np.uint8)
    #im = cv2.imread(img)
    fig = plt.figure()
    ax1 = fig.add_subplot(111,)
    category = categories
    gt_boxes = deltas
    ax1.imshow(im)
    #gt_boxes = gt_boxes[:4]
    for r in range(0,len(gt_boxes)):
        ax1.add_patch(patches.Rectangle((gt_boxes[r,0],gt_boxes[r,1]),gt_boxes[r,2]-gt_boxes[r,0],gt_boxes[r,3]-gt_boxes[r,1],1,linewidth=1,edgecolor='r',facecolor='none'))
        plt.text(gt_boxes[r,0],gt_boxes[r,1],category[r], color = 'black')
    plt.show()

In [223]:
def plotTest():
    count=0
    for files in os.listdir(pred_img_path):
        imgfile = pred_img_path + '\\' + files
        anno = pred_anno_path + '\\' + files[:-4] + '.txt'
        img_, scw, sch = preprocess_(imgfile)
        categories, boxes = predictImages(img_,model_rcnn,rois)
        plotPredicted(img_, boxes, categories)

In [194]:
def calc(boxes,gt_boxes):
    avg = 0.0
    for i in range(0,len(boxes)):
        box_area = (boxes[i, 2] - boxes[i, 0] + 1) * (boxes[i, 3] - boxes[i, 1] + 1)
        gt_area = (gt_boxes[i, 2] - gt_boxes[i, 0] + 1) * (gt_boxes[i, 3] - gt_boxes[i, 1] + 1)
        iw = (min(gt_boxes[i, 2], boxes[i, 2]) - max(gt_boxes[i, 0], boxes[i, 0]) + 1)
        ih = (min(gt_boxes[i, 3], boxes[i, 3]) - max(gt_boxes[i, 1], boxes[i, 1]) + 1)
        ua = box_area+gt_area-(iw*ih)
        avg = avg + ((iw*ih)/ua)
    avg = avg/len(boxes)
    return avg

In [218]:
value =[]
count=0
for files in os.listdir(pred_img_path):
    imgfile = pred_img_path + '\\' + files
    anno = pred_anno_path + '\\' + files[:-4] + '.txt'
    img_, scw, sch = preprocess_(imgfile)
    categories, boxes = predictImages(img_,model_rcnn,rois)
    cats, gt_boxes = parse_(file+filename,scw,sch)
    value.append(calc(boxes,gt_boxes)
    count = count+1
value = value/count

In [46]:
model_rcnn.save('rcnn_weightsUpdated.h5')